# MÔ TẢ CÁC BƯỚC TRONG ACTOR-CRITIC VỚI MCTS
# Actor-Critic kết hợp Monte Carlo Tree Search (MCTS) là một cách tiếp cận kết hợp giữa học tăng cường và tìm kiếm cây. Trong thuật toán này:
# 1. **Actor (Chính sách)**: Xác định xác suất chọn các hành động từ trạng thái.
# 2. **Critic (Giá trị)**: Ước lượng giá trị trạng thái để hỗ trợ Actor điều chỉnh chính sách.
# 3. **MCTS**: Mô phỏng các hành động tiềm năng từ trạng thái hiện tại, giúp lựa chọn hành động tối ưu bằng cách sử dụng thông tin của Actor và Critic.
# 4. **Entropy Regularization**: Tăng độ đa dạng trong các hành động được chọn để khuyến khích khám phá.
# **Các bước chính trong thuật toán:**
# 1. **Khởi tạo**: Tạo môi trường, thiết lập mô hình Actor-Critic và các thông số.
# 2. **MCTS**: Mô phỏng nhiều lần để xác định hành động tốt nhất tại mỗi trạng thái.
# 3. **Thu thập dữ liệu**: Chạy tập huấn luyện, ghi nhận phần thưởng và hành động.
# 4. **Tính Advantage và Returns**: Sử dụng giá trị Critic và phần thưởng để tính lợi thế.
# 5. **Cập nhật mô hình**: Tối ưu hóa Actor và Critic dựa trên mất mát.
# 6. **Theo dõi kết quả**: Lưu lại phần thưởng trung bình để đánh giá hiệu suất.


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import gym
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [ ]:
# ==================== CẤU HÌNH THUẬT TOÁN ====================
# Khởi tạo các tham số chính để thực hiện Actor-Critic kết hợp với MCTS
seed = 42  # Seed để tái hiện kết quả
gamma = 0.99  # Hệ số chiết khấu, quyết định tầm quan trọng của phần thưởng tương lai
max_steps_per_episode = 1000  # Số bước tối đa cho mỗi tập huấn luyện
env = gym.make("CartPole-v1")  # Tạo môi trường CartPole
env.seed(seed)  # Gán seed cho môi trường
eps = np.finfo(np.float32).eps.item()  # Giá trị epsilon nhỏ để tránh chia cho 0 trong tính toán

# ==================== XÂY DỰNG MÔ HÌNH ACTOR-CRITIC ====================
# Thiết lập cấu trúc mô hình neural mạng cho Actor và Critic
num_inputs = 4  # Số lượng đầu vào (tính năng trạng thái của môi trường)
num_actions = 2  # Số lượng hành động có thể thực hiện
num_hidden = 256  # Số lượng neuron trong tầng ẩn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

In [ ]:
# Xây dựng mô hình Actor-Critic
# Tạo đầu vào của mạng
inputs = layers.Input(shape=(num_inputs,))
# Tầng ẩn 1 với kích hoạt ReLU
common = layers.Dense(num_hidden, activation="relu")(inputs)
# Tầng ẩn 2 với kích hoạt ReLU
common = layers.Dense(num_hidden, activation="relu")(common)
# Tầng đầu ra cho Actor: trả về xác suất các hành động
action = layers.Dense(num_actions, activation="softmax")(common)
# Tầng đầu ra cho Critic: trả về giá trị trạng thái
critic = layers.Dense(1)(common)
# Kết hợp các tầng thành một mô hình
model = keras.Model(inputs=inputs, outputs=[action, critic])

# ==================== TỐI ƯU HÓA MÔ HÌNH ====================
# Sử dụng Adam optimizer với learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)
# Sử dụng hàm mất mát Huber để giảm ảnh hưởng của ngoại lệ trong Critic
huber_loss = keras.losses.Huber()

# ==================== BIẾN LƯU TRỮ LỊCH SỬ ====================
# Lưu trữ lịch sử xác suất hành động, giá trị Critic và phần thưởng
action_probs_history = []  # Xác suất của các hành động đã thực hiện
critic_value_history = []  # Giá trị trạng thái từ Critic
rewards_history = []  # Phần thưởng nhận được
running_reward = 0  # Phần thưởng trung bình chạy qua các tập
episode_count = 0  # Bộ đếm số tập huấn luyện
max_episodes = 1000  # Số tập tối đa
entropy_beta = 0.05  # Hệ số cho entropy regularization
num_simulations = 50  # Số lần mô phỏng trong MCTS
running_rewards = []  # Lưu phần thưởng trung bình

In [ ]:
# Mô hình hóa một nút trong cây tìm kiếm MCTS
class MCTSNode:
    def __init__(self, state, parent=None, prior_prob=1.0):
        self.state = state  # Trạng thái tương ứng với nút
        self.parent = parent  # Nút cha
        self.children = {}  # Danh sách các nút con
        self.visits = 0  # Số lần nút này được thăm
        self.value = 0  # Giá trị trung bình của nút
        self.prior_prob = prior_prob  # Xác suất ban đầu của hành động tương ứng

    def is_fully_expanded(self):
        # Kiểm tra xem nút đã mở rộng tất cả các hành động hay chưa
        return len(self.children) == num_actions

    def best_child(self, c_puct=1.0):
        # Tìm nút con tốt nhất dựa trên giá trị UCB (Upper Confidence Bound)
        return max(
            self.children.items(),
            key=lambda child: child[1].value / (1 + child[1].visits) +
                              c_puct * child[1].prior_prob * np.sqrt(self.visits) / (1 + child[1].visits)
        )[1]

    def expand(self, action_probs):
        # Mở rộng nút bằng cách thêm các hành động mới chưa được thêm vào cây
        for action, prob in enumerate(action_probs):
            if action not in self.children:
                self.children[action] = MCTSNode(self.state, parent=self, prior_prob=prob)

    def backpropagate(self, reward):
        # Lan truyền phần thưởng ngược từ nút hiện tại về gốc
        self.value += reward
        self.visits += 1
        if self.parent:
            self.parent.backpropagate(reward)


In [ ]:
# Hàm thực hiện thuật toán Monte Carlo Tree Search
def run_mcts(root, model, num_simulations=num_simulations, c_puct=1.0):
    for _ in range(num_simulations):
        node = root  # Bắt đầu từ nút gốc
        # Duyệt cây cho đến khi gặp nút chưa mở rộng
        while node.is_fully_expanded() and node.children:
            node = node.best_child(c_puct)
        # Chuyển trạng thái nút thành tensor để dự đoán xác suất hành động và giá trị Critic
        state_tensor = tf.convert_to_tensor(node.state, dtype=tf.float32)
        state_tensor = tf.expand_dims(state_tensor, 0)
        action_probs, critic_value = model(state_tensor)
        action_probs = action_probs.numpy().squeeze()
        # Mở rộng cây nếu nút hiện tại chưa được mở rộng
        if not node.is_fully_expanded():
            node.expand(action_probs)
        # Nhận giá trị phần thưởng từ Critic
        reward = critic_value.numpy()[0, 0]
        # Lan truyền phần thưởng ngược lên cây
        node.backpropagate(reward)
    # Trả về hành động với lượt thăm cao nhất
    return max(root.children.items(), key=lambda child: child[1].visits)[0]


# Loss với Entropy Regularization
def compute_loss(action_probs_history, critic_value_history, returns, entropy_beta):
    actor_losses, critic_losses = [], []
    entropy = 0
    for log_prob, value, ret in zip(action_probs_history, critic_value_history, returns):
        advantage = ret - value
        actor_losses.append(-log_prob * advantage)
        critic_losses.append(huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0)))
        entropy += -tf.reduce_sum(action_probs_history[-1] * tf.math.log(action_probs_history[-1] + eps))
    total_loss = sum(actor_losses) + sum(critic_losses) - entropy_beta * entropy
    return total_loss

In [ ]:
# Vòng huấn luyện Actor-Critic
# Huấn luyện mô hình Actor-Critic với MCTS
while episode_count < max_episodes:
    state = env.reset()  # Đặt lại môi trường
    episode_reward = 0  # Tổng phần thưởng trong tập
    with tf.GradientTape() as tape:  # Gradient Tape để theo dõi gradient
        for timestep in range(1, max_steps_per_episode + 1):
            # Chuyển trạng thái thành tensor
            state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs, critic_value = model(state_tensor)  # Lấy dự đoán từ Actor và Critic
            critic_value_history.append(critic_value[0, 0])  # Lưu giá trị Critic
            root = MCTSNode(state)  # Tạo nút gốc MCTS với trạng thái hiện tại
            root.expand(action_probs.numpy().squeeze())  # Mở rộng nút gốc
            action = run_mcts(root, model)  # Chạy MCTS để chọn hành động
            action_probs_history.append(action_probs[0, action])  # Lưu xác suất hành động
            state, reward, done, _ = env.step(action)  # Thực hiện hành động
            rewards_history.append(reward)  # Lưu phần thưởng
            episode_reward += reward  # Cộng phần thưởng vào tổng tập
            if done:  # Dừng tập nếu trạng thái kết thúc
                break

        # Tính toán Returns và Advantage
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)  # Chuẩn hóa Returns
        returns = returns.tolist()

        # Tính mất mát và cập nhật mô hình
        loss_value = compute_loss(action_probs_history, critic_value_history, returns, entropy_beta)
        grads = tape.gradient(loss_value, model.trainable_variables)  # Lấy gradient
        grads, _ = tf.clip_by_global_norm(grads, 1.0)  # Cắt gradient để tránh quá lớn
        optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Cập nhật mô hình

        # Dọn lịch sử sau mỗi tập
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    episode_count += 1
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
    running_rewards.append(running_reward)

    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, Running Reward: {running_reward:.2f}")
    if running_reward > 475:
        print(f"Solved at episode {episode_count}!")
        break

if episode_count == max_episodes:
    print(f"Reached the maximum episode limit of {max_episodes}. Training stopped.")

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 10, Running Reward: 3.75
Episode: 20, Running Reward: 6.10
Episode: 30, Running Reward: 7.60
Episode: 40, Running Reward: 8.27
Episode: 50, Running Reward: 8.69
Episode: 60, Running Reward: 8.98
Episode: 70, Running Reward: 9.22
Episode: 80, Running Reward: 9.30
Episode: 90, Running Reward: 9.24
Episode: 100, Running Reward: 9.40
Episode: 110, Running Reward: 9.56
Episode: 120, Running Reward: 9.42
Episode: 130, Running Reward: 9.47
Episode: 140, Running Reward: 9.53
Episode: 150, Running Reward: 9.44
Episode: 160, Running Reward: 9.32
Episode: 170, Running Reward: 9.41
Episode: 180, Running Reward: 9.30
Episode: 190, Running Reward: 9.49
Episode: 200, Running Reward: 9.60
Episode: 210, Running Reward: 9.53
Episode: 220, Running Reward: 9.42
Episode: 230, Running Reward: 9.41
Episode: 240, Running Reward: 9.31
Episode: 250, Running Reward: 9.20
Episode: 260, Running Reward: 9.21
Episode: 270, Running Reward: 9.23
Episode: 280, Running Reward: 9.23
Episode: 290, Running Reward: